# High-Utility Action Rules Mining Example

### Import Module

In [1]:
from action_rules import ActionRules

### Import Data

The aim of this example is to find actions that increase the probability that employees do not leave the company.

In [2]:
import pandas as pd
# Data
data = pd.read_csv("data/attrition.csv")
data

,TID,Department,Salary,Attrition
0,1,Sales,Medium,False
1,2,R&D,Medium,False
2,3,R&D,Medium,True
3,4,R&D,Medium,True
4,5,Sales,Low,False
5,6,R&D,High,False
6,7,R&D,High,False
7,8,R&D,High,True


## Utility Tables

In [3]:
intrinsic_table = {
    ('Salary', 'Low'): -300.0,
    ('Salary', 'Medium'): -500.0,
    ('Salary', 'High'): -1000.0,
    ('Attrition', 'False'): 700.0,
    ('Attrition', 'True'): 0.0,
}
# cost per transition
transition_table = {
    ('Salary', 'Low', 'Medium'): -2.0,
    ('Salary', 'Low', 'High'): -4.0,
    ('Salary', 'Medium', 'High'): -2.5,
}

### Explanation of Utility Tables

The **intrinsic_utility_table** and **transition_utility_table** are two key inputs for high-utility action rule mining. They define, respectively, the inherent value (or cost) of having a particular attribute value and the cost (or benefit) of changing an attribute value from one state to another.

#### intrinsic_utility_table

The intrinsic utility table assigns a utility value to each attribute–value pair. This value represents the inherent "worth" or "cost" associated with that specific state of an attribute. Typically:

- **Positive utility values** indicate a benefit or gain.
- **Negative utility values** indicate a cost or penalty.

For example:
- `('Salary', 'Low'): -300.0` means that having a low salary incurs a cost of 300 units.
- `('Salary', 'Medium'): -500.0` means that a medium salary incurs a cost of 500 units.
- `('Salary', 'High'): -1000.0` means that a high salary incurs a cost of 1000 units.
- `('Attrition', 'False'): 700.0` means that the condition "not leaving the company" (Attrition False) has a benefit of 700 units.
- `('Attrition', 'True'): 0.0` means that the condition "leaving the company" (Attrition True) has no benefit (or a cost of 0).

In high-utility action rule mining, these intrinsic utilities are used to compute the overall utility of a rule before considering any changes (transitions).

#### transition_utility_table

The transition utility table specifies the cost or benefit of changing an attribute’s value from one state to another. These values capture the incremental effect (or cost) associated with a particular change in an attribute's state. For example:

- `('Salary', 'Low', 'Medium'): -2.0` means that the cost of increasing a salary from Low to Medium is 2 units.
- `('Salary', 'Low', 'High'): -4.0` means that the cost of increasing a salary from Low to High is 4 units.
- `('Salary', 'Medium', 'High'): -2.5` means that the cost of increasing a salary from Medium to High is 2.5 units.

These transition utilities are combined with the intrinsic utilities of the target states to determine the overall benefit (or cost) of performing an action that changes an attribute’s value.

---

Together, these tables allow the system to compute both:
- **Base (Intrinsic) Utilities:** The inherent worth of a rule based on the states of its attributes.
- **Transition Gains/Costs:** The additional effects of changing an attribute’s state, including any associated cost or benefit for modifying the target attribute.

When combined with support and confidence measures from the mined rules, these utility values help determine which action rules are most profitable or cost-effective.


### Initialize ActionRules Miner with Parameters

Utility parameters: intrinsic_utility_table, transition_utility_table

In [4]:
# Parameters
stable_attributes = ['Department']
flexible_attributes = ['Salary']
target = 'Attrition'
min_stable_attributes = 1
min_flexible_attributes = 1 #min 1
min_undesired_support = 2
min_undesired_confidence = 0.6 #min 0.5
min_desired_support = 2
min_desired_confidence = 0.6 #min 0.5
undesired_state = 'True'
desired_state = 'False'
# Action Rules Mining
action_rules = ActionRules(
    min_stable_attributes=min_stable_attributes,
    min_flexible_attributes=min_flexible_attributes,
    min_undesired_support=min_undesired_support,
    min_undesired_confidence=min_undesired_confidence,
    min_desired_support=min_desired_support,
    min_desired_confidence=min_desired_confidence,
    intrinsic_utility_table=intrinsic_table,
    transition_utility_table=transition_table,
)

### Train the Model

In [5]:
action_rules.fit(
    data=data,
    stable_attributes=stable_attributes,
    flexible_attributes=flexible_attributes,
    target=target,
    target_undesired_state=undesired_state,
    target_desired_state=desired_state,
    use_sparse_matrix=False,
    use_gpu=False,
)

### Print Action Rules with Utility Metrics

In [6]:
len(action_rules.get_rules().get_ar_notation())

1

In [7]:
# Print rules
for action_rule in action_rules.get_rules().get_ar_notation():
    print(action_rule)

[(Department: R&D) ∧ (Salary: Medium → High)] ⇒ [Attrition: True → False], support of undesired part: 2, confidence of undesired part: 0.6666666666666666, support of desired part: 2, confidence of desired part: 0.6666666666666666, support: 2, confidence: 0.4444444444444444, uplift: 0.12499999999999997, max_rule_gain: 197.5, realistic_rule_gain: -269.16666666666674, realistic_dataset_gain: -807.5000000000002


In [8]:
for action_rule in action_rules.get_rules().get_pretty_ar_notation():
    print(action_rule)

If attribute 'Department' is 'R&D', attribute 'Salary' value 'Medium' is changed to 'High', then 'Attrition' value 'True' is changed to 'False with support: 2, confidence: 0.4444444444444444, uplift: 0.12499999999999997, support of undesired part: 2, confidence of undesired part: 0.6666666666666666, support of desired part: 2, confidence of desired part: 0.6666666666666666, max_rule_gain: 197.5, realistic_rule_gain: -269.16666666666674, realistic_dataset_gain: -807.5000000000002.


In [9]:
export = action_rules.get_rules().get_export_notation()
print(export)

[{"stable": [{"attribute": "Department", "value": "R&D"}], "flexible": [{"attribute": "Salary", "undesired": "Medium", "desired": "High"}], "target": {"attribute": "Attrition", "undesired": "True", "desired": "False"}, "support of undesired part": 2, "confidence of undesired part": 0.6666666666666666, "support of desired part": 2, "confidence of desired part": 0.6666666666666666, "uplift": 0.12499999999999997, "support": 2, "confidence": 0.4444444444444444, "max_rule_gain": 197.5, "realistic_rule_gain": -269, "realistic_dataset_gain": -807.5000000000002}]
